#### 0) 데이터 확인, 전처리 - 필요없음/완료 파일 드라이브에 업로드 완료!

In [1]:
# None 값이 있는지 확인
import pandas as pd
import json

file_path = "../data/scraping/all_book_data_ver3.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON 데이터가 리스트인지 딕셔너리인지 확인 후 변환
if isinstance(data, list):
    df = pd.DataFrame(data)
elif isinstance(data, dict):  # 딕셔너리 형태이면 변환 (예: {key: list})
    df = pd.DataFrame.from_dict(data)

# None 값 확인
none_count = df.isna().sum()

print(none_count)

title                0
description          0
key_sentences        0
author               0
publisher            0
isbn13              13
img_url              0
publication_date    14
dtype: int64


중복 null 13개 + pulication_date 1개 => 총 14개 null

In [2]:
import pandas as pd
import json

# JSON 파일 로드
file_path = "../data/scraping/all_book_data_ver3.json"  # 파일 경로 설정
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON 데이터를 DataFrame으로 변환
if isinstance(data, list):
    df = pd.DataFrame(data)
elif isinstance(data, dict):  # 딕셔너리 형태이면 변환
    df = pd.DataFrame.from_dict(data)

# 📌 'ISBN' 접두사 제거
df["isbn13"] = df["isbn13"].astype(str).str.replace(r'^ISBN', '', regex=True)

# 📌 이미지 URL 변환 (JSON 데이터가 로드된 후 직접 수정)
df["img_url"] = df["img_url"].astype(str).apply(lambda x: x.replace("\\/", "/"))

# 📌 `isbn13` 값이 숫자가 아닌 경우 삭제
df = df[df["isbn13"].str.isdigit()]  # 숫자로만 이루어진 isbn13 값만 유지

# 📌 'isbn13' 또는 'publication_date' 값이 None인 행 삭제
df.dropna(subset=['isbn13', 'publication_date'], inplace=True)

# 📌 모든 컬럼에서 하나라도 None 값이 있는 행 삭제
df.dropna(inplace=True)

# 📌 수정된 데이터를 JSON으로 저장
output_path = "../data/scraping/all_book_data_ver3_cleaned.json"
df.to_json(output_path, orient="records", force_ascii=False, indent=4)

print(f"✅ 수정 완료! 저장된 파일: {output_path}")

✅ 수정 완료! 저장된 파일: ../data/scraping/all_book_data_ver3_cleaned.json


In [4]:
import json
import os
import re

# 📌 JSON 파일 경로
json_file_path = "../data/scraping/all_book_data_ver3_cleaned.json"
output_json_path = "../data/scraping/all_book_data_ver3_filtered.json"

# JSON 파일 로드
with open(json_file_path, "r", encoding="utf-8") as f:
    book_data = json.load(f)


filtered_books = []
for book in book_data:
    if "key_sentences" in book and "isbn13" in book:
        if isinstance(book["key_sentences"], str):
            # 1️⃣ '중에서' 이후의 내용 제거
            book["key_sentences"] = re.split(r'중에서', book["key_sentences"], maxsplit=1)[0].strip() + '중에서' if '중에서' in book["key_sentences"] else book["key_sentences"]
            # 2️⃣ '--- p.숫자' 등의 구분자를 기준으로 분리
            book["key_sentences"] = re.split(r'---', book["key_sentences"], maxsplit=1)[0].strip()

            filtered_books.append({
                "isbn" : book["isbn13"],
                "key_sentences": book["key_sentences"]
            })

            
# Json 파일 저장
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print("핵심 문장 필터링 완료! 결과 저장됨: ", output_json_path)

핵심 문장 필터링 완료! 결과 저장됨:  ../data/scraping/all_book_data_ver3_filtered.json


#### 1) 데이터베이스가 테이블이 생성되고 json 파일이랑 DB랑 연결
- 구글 드라이브에서 all_book_data_ver3_cleaned.json파일 다운로드

In [11]:
# from IPython.core.history import HistoryManager

# history_manager = HistoryManager()
# history_manager.reset()

In [0]:
# Python 경로 추가
import sys
import os
import json
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from app.database.connection import DatabaseSettings, create_connection
from app.services.data_ingestion_service import create_tables
from app.services.data_insert_DB import insert_books

# 데이터베이스 연결 설정
db_settings = DatabaseSettings()
conn = create_connection(db_settings)

if conn:
    # 테이블 생성
    create_tables(conn)
    print('success')

    # JSON 파일 경로
    json_file_path = "../data/scraping/all_book_data_ver3_cleaned.json"

    # JSON 데이터를 테이블에 삽입
    insert_books(conn, json_file_path)

    # 데이터 조회 테스트
    def test_connection(conn):
        """테이블 데이터 조회 테스트"""
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM books;")
            rows = cursor.fetchall()
            if rows:
                for row in rows:
                    print(row)
            else:
                print("No data found in 'books' table.")
        except Exception as e:
            print("Error during connection test:", e)
        finally:
            cursor.close()

    test_connection(conn)

    # 연결 종료
    conn.close()
    print("Database connection closed.")

Database settings successfully!


In [9]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 출력
print(os.path.exists(json_file))  # 파일 존재 여부 확인

/Users/jin0/Project/Book-reccomendation/test


NameError: name 'json_file' is not defined

In [10]:
import os
print(os.getcwd())  # 현재 디렉토리 출력

/Users/jin0/Project/Book-reccomendation/test


In [0]:
# 생성문자와 해시태그 데이터 적재
import os
import sys

# 현재 작업 디렉토리 확인 및 프로젝트 경로 추가
current_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(current_dir, "../"))  # 현재 디렉토리에서 상위 두 단계 경로를 추적
sys.path.append(project_dir)

# 데이터 삽입 함수 import
from app.services.data_insert_DB import insert_hashtag_message_data
from app.database.connection import DatabaseSettings, create_connection

def main():
    settings = DatabaseSettings()  # DB 설정 객체 생성
    conn = create_connection(settings)  # DB 연결
    
    if conn:
        try:
            with conn.cursor() as cursor:
                json_file = "../data/scraping/all_book_data_ver3_cleaned.json"  # 파일명 안전하게 수정
                insert_hashtag_message_data(cursor, json_file)
            conn.commit()
            print("✅ 데이터 삽입 완료!")
        except Exception as e:
            conn.rollback()
            print(f"⚠️ 오류 발생: {e}")
        finally:
            conn.close()
            print("📌 Database connection closed.")

if __name__ == "__main__":
    main()

In [ ]:
from app.services.data_insert_DB import insert_hashtag_message_data
from app.database.connection import DatabaseSettings, create_connection

